Food Sales Predictions Project

John Lynn

.

This is my second submission of this assignment. The comment on my Feedback form for this assignment said, "Find and fix any inconsistent categories of data"

I had already done that and worked with TA_Nicole to double-check my original submission, but we were unable to find any inconsistencies in my first file.

To make it more clear, I changed this file to run the 'value_counts' on each of the Dtype object columns individually instead of all together in one step.

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Part 1

In [57]:
import pandas as pd
filename = '/content/sales_predictions.csv'

In [58]:
sales_predictions_df = pd.read_csv(filename)
sales_predictions_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052



Part 2

In [59]:
sales_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [60]:
# Search for duplicate data
sales_predictions_df.duplicated().any()

False

Before I get into dealing with the null values, I want to clean up any inconsistencies in the columns

In [61]:
sales_predictions_df[['Item_Identifier']].value_counts()

Item_Identifier
FDW13              10
FDG33              10
NCQ06               9
FDF56               9
FDX20               9
                   ..
FDC23               1
FDQ60               1
FDK57               1
FDO33               1
DRF48               1
Length: 1559, dtype: int64

In [62]:
sales_predictions_df[['Item_Fat_Content']].value_counts()

Item_Fat_Content
Low Fat             5089
Regular             2889
LF                   316
reg                  117
low fat              112
dtype: int64

In [63]:
sales_predictions_df[['Item_Type']].value_counts()

Item_Type            
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
dtype: int64

In [64]:
sales_predictions_df[['Outlet_Identifier']].value_counts()

Outlet_Identifier
OUT027               935
OUT013               932
OUT049               930
OUT046               930
OUT035               930
OUT045               929
OUT018               928
OUT017               926
OUT010               555
OUT019               528
dtype: int64

In [65]:
sales_predictions_df[['Outlet_Size']].value_counts()

Outlet_Size
Medium         2793
Small          2388
High            932
dtype: int64

In [66]:
sales_predictions_df[['Outlet_Location_Type']].value_counts()

Outlet_Location_Type
Tier 3                  3350
Tier 2                  2785
Tier 1                  2388
dtype: int64

In [67]:
sales_predictions_df[['Outlet_Type']].value_counts()

Outlet_Type      
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
dtype: int64

In [68]:
# The Item_Fat_Content column has differences in the labeling, so I need to make them consistent
sales_predictions_df['Item_Fat_Content'].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'}, inplace=True)

In [70]:
# Running this again to show the labels were corrected
sales_predictions_df[['Item_Fat_Content']].value_counts()

Item_Fat_Content
Low Fat             5517
Regular             3006
dtype: int64

In [71]:
# Clarify the missing values
sales_predictions_df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [72]:
# Isolate all the rows missing data for the Outlet_Size column
sales_predictions_df.loc[sales_predictions_df['Outlet_Size'].isna()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350
25,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,NaN,Tier 2,Supermarket Type1,838.9080
28,FDE51,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,NaN,Tier 3,Grocery Store,178.4344
...,...,...,...,...,...,...,...,...,...,...,...,...
8502,NCH43,8.420,Low Fat,0.070712,Household,216.4192,OUT045,2002,NaN,Tier 2,Supermarket Type1,3020.0688
8508,FDW31,11.350,Regular,0.043246,Fruits and Vegetables,199.4742,OUT045,2002,NaN,Tier 2,Supermarket Type1,2587.9646
8509,FDG45,8.100,Low Fat,0.214306,Fruits and Vegetables,213.9902,OUT010,1998,NaN,Tier 3,Grocery Store,424.7804
8514,FDA01,15.000,Regular,0.054489,Canned,57.5904,OUT045,2002,NaN,Tier 2,Supermarket Type1,468.7232


All of the rows with null values in the Outlet_Size colum have either a Tier 2 Supermarket Type 1 or Tier 3 Grocery Store  Outlet_Location_Type and Outlet_Type

All non-null Tier 2 Supermarket Type 1 and all Grocery Store instances in the DataFrame have an Outlet_Size of Small, so I am going to replace all of the null values in the 'Outlet_Size' column with 'Small'

In [73]:
# Replace nulls with Small
sales_predictions_df['Outlet_Size'].fillna(value='Small', inplace=True)

In [74]:
# Run the .isnull again to make sure the null values of the Outlet_Size column are 0
sales_predictions_df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

The 1463 null values in the Item_Weight column make up 17% of that column.

I think dropping that percentage of rows from the dataset might negatively impact the rest of the data in the other columns.

I am going to get the mean of the Item_Weight column and fill the null values with that average value.

In [75]:
# First running the .mean for the entire column, including the null values
sales_predictions_df['Item_Weight'].mean()

12.857645184136183

In [76]:
# Use the .fillna to impute the column average over the null values
sales_predictions_df.fillna(12.857645184136183, inplace = True)
# Run the .info again to check the Non-Null Count again
sales_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [77]:
# Run the .mean for the column again to make sure it looks right
sales_predictions_df['Item_Weight'].mean()

12.857645184136409

Summary Statistics of Numerical Columns

In [78]:
# Minimums
sales_predictions_df[['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales']].min()

Item_Weight                     4.555
Item_Visibility                 0.000
Item_MRP                       31.290
Outlet_Establishment_Year    1985.000
Item_Outlet_Sales              33.290
dtype: float64

In [79]:
# Maximums
sales_predictions_df[['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales']].max()

Item_Weight                     21.350000
Item_Visibility                  0.328391
Item_MRP                       266.888400
Outlet_Establishment_Year     2009.000000
Item_Outlet_Sales            13086.964800
dtype: float64

In [80]:
# Mean
sales_predictions_df[['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales']].mean()

Item_Weight                    12.857645
Item_Visibility                 0.066132
Item_MRP                      140.992782
Outlet_Establishment_Year    1997.831867
Item_Outlet_Sales            2181.288914
dtype: float64